In [160]:
import functools
import itertools

import numpy as np

from pycobyla.gcobyla import GCobyla

import tests.test_custom as tc
from tests.nb import utils

In [161]:
%matplotlib widget

import matplotlib.pyplot as plt
from matplotlib import cm

def ax_plot_simplex(ax, optimal_vertex, sim):
    simplex = np.array((optimal_vertex, *(sim + optimal_vertex), optimal_vertex))
    ax.plot(*optimal_vertex, color='limegreen', marker='o')
    ax.plot(simplex[1:-1,0], simplex[1:-1,1], '.', color='blue', marker='o')
    ax.plot(simplex[...,0], simplex[...,1], '-.', color='gray', lw=2)
    

def ax_plot_track(opt, ax, target, from_points=0, to_points=-1, plot_simplex=False, plot_trure=False, aspect=False, track_color='red'):
    track = opt.track[from_points:to_points]
    ax.plot(track[:, 0], track[:, 1], linestyle='-', color=track_color, marker='o')

    link_best = np.array((track[-1], target))
    ax.plot(link_best[:, 0], link_best[:, 1], linestyle=':', color=track_color)
    ax.plot(target[0], target[1], color='limegreen', marker='*')
        
    if plot_simplex:
        #simplex = np.array((opt.optimal_vertex, *(opt.sim + opt.optimal_vertex), opt.optimal_vertex))
        ax_plot_simplex(ax, opt.optimal_vertex, opt.sim)
            
    if plot_trure:
        trust_region = plt.Circle(opt.optimal_vertex, opt.rho, color='khaki', fill=True, alpha=0.5)
        ax.add_patch(trust_region)
        
    if aspect:
        ax.set_aspect('equal')
        
def ax_plot_map(fig, ax, opt, border, nsamples=100):
    X = np.linspace(-border, border, nsamples)
    xv, yv = np.meshgrid(X, X)
    zv = np.array(tuple(-opt.F(xy) for xy in itertools.product(X, X)))
    zv = zv.reshape(xv.shape)
    
    cs = ax.contourf(xv, yv, zv, cmap=cm.magma)
    #cbar = fig.colorbar(cs)
    
    #ax.plot(np.array((-border, border)), np.array((0, 0)), linestyle='-.', color='royalblue')
    #ax.plot(np.array((0, 0)), np.array((-border, border)), linestyle='-.', color='royalblue')


In [162]:
def gaussian_optimizer(start_x, mu=None, sig=None, A=-1):
    mu = np.zeros(2) if mu is None else mu
    sig = np.ones(2) if sig is None else sig
    
    F = functools.partial(tc.gaussian, mu=mu, sig=sig, A=A)
    C = ()
    
    opt = GCobyla(start_x, F, C, rhobeg=0.5, rhoend=1e-12)
    return opt

In [163]:
plt.close('all')
fig, axes = plt.subplots(2, 3, figsize=(12, 9))
mu = (0, 0)

gg = functools.partial(gaussian_optimizer, mu=mu, A=-1)

rsc = {
    axes[0,0]: ((.5**.5,.5**.5),'a)'),
    axes[0,1]: ((0,1), 'b)'),
    axes[0,2]: ((-.5**.5,.5**.5), 'c)'),
    axes[1,0]: ((-.5**.5,-.5**.5), 'd)'),
    axes[1,1]: ((0,-1), 'e)'),
    axes[1,2]: ((.5**.5,-.5**.5),'f)'),
}
    

for ax, data in rsc.items():
    start_x, caption = data
    opt = gg(start_x=start_x)
    steps_it = (step for step in opt.g_run() if step in {GCobyla.BEFORE_REVIEW_CURRENT_SIMPLEX_CHECKPOINT, GCobyla.BEFORE_GENERATE_X_START_CHECKPOINT})
    for _ in steps_it: pass

    ax.set_xlabel(f'{caption} start x at ({start_x[0]:.2f}, {start_x[1]:.2f})')
    ax_plot_track(opt, ax, mu,  aspect=True, track_color='darkred')
    ax_plot_map(fig, ax, opt, border=2, nsamples=50)
    
plt.savefig('ex1-1.png', dpi=120, bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Una normal amb restriccions

In [164]:
def gaussian_optimizer(start_x, border=1, mu=None, sig=None, A=-1):
    mu = np.zeros(2) if mu is None else mu
    sig = np.ones(2) if sig is None else sig
    
    F = functools.partial(tc.gaussian, mu=mu, sig=sig, A=A)
    c1 = lambda x: border - x[0]
    c2 = lambda x: border + x[0]
    c3 = lambda x: border - x[1]
    c4 = lambda x: border + x[1]
    C = (c1, c2, c3, c4)
    
    opt = GCobyla(start_x, F, C, rhobeg=0.5, rhoend=1e-12)
    return opt

In [174]:
plt.close('all')
fig, axes = plt.subplots(2, 3, figsize=(12, 9))
border = .5
mu = (0, 0)

gg = functools.partial(gaussian_optimizer, border=border, mu=mu, A=-1)

rsc = {
    axes[0,0]: ((.5**.5,.5**.5),'a)'),
    axes[0,1]: ((0,1), 'b)'),
    axes[0,2]: ((-.5**.5,.5**.5), 'c)'),
    axes[1,0]: ((-.5**.5,-.5**.5), 'd)'),
    axes[1,1]: ((0,-1), 'e)'),
    axes[1,2]: ((.5**.5,-.5**.5),'f)'),
}
    

for ax, data in rsc.items():
    start_x, caption = data
    opt = gg(start_x=start_x)
    steps_it = (step for step in opt.g_run() if step in {GCobyla.BEFORE_REVIEW_CURRENT_SIMPLEX_CHECKPOINT, GCobyla.BEFORE_GENERATE_X_START_CHECKPOINT})
    for _ in steps_it: pass

    ax.set_xlabel(f'{caption} start x at ({start_x[0]:.2f}, {start_x[1]:.2f})')
    ax_plot_track(opt, ax, mu,  aspect=True, track_color='darkred')
    ax_plot_map(fig, ax, opt, border=2, nsamples=50)
    
    region = np.array(((border, border), (-border,border), (-border, -border), (border, -border), (border, border)))
    ax.plot(region[..., 0], region[..., 1], linestyle='-.', color='black')
    
plt.savefig('mem/rsc/ex1-2.png', dpi=120, bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [175]:
plt.close('all')
fig, axes = plt.subplots(2, 3, figsize=(12, 9))
border = 1
mu = (0, 0)

gg = functools.partial(gaussian_optimizer, mu=mu, A=-1)

rsc = {
    axes[0,0]: ((.5**.5,.5**.5), 'a)'),
    axes[0,1]: ((.5**.5,.5**.5), 'b)'),
    axes[0,2]: ((.5**.5,.5**.5), 'c)'),
    axes[1,0]: ((.5**.5,.5**.5), 'd)'),
    axes[1,1]: ((.5**.5,.5**.5), 'e)'),
    axes[1,2]: ((.5**.5,.5**.5), 'f)'),
}
    

for ax, data in rsc.items():
    start_x, caption = data
    opt = gg(start_x=start_x)
    steps_it = (step for step in opt.g_run() if step in {GCobyla.BEFORE_REVIEW_CURRENT_SIMPLEX_CHECKPOINT, GCobyla.BEFORE_GENERATE_X_START_CHECKPOINT})
    for _ in steps_it: pass

    ax.set_xlabel(f'{caption} start x at ({start_x[0]:.2f}, {start_x[1]:.2f}), length side {2*border:.2f}')
    ax_plot_track(opt, ax, mu,  aspect=True, track_color='darkred')
    ax_plot_map(fig, ax, opt, border=2, nsamples=50)
    
    region = np.array(((border, border), (-border,border), (-border, -border), (border, -border), (border, border)))
    ax.plot(region[..., 0], region[..., 1], linestyle='-.', color='black')
    border /= 2
plt.savefig('mem/rsc/ex1-3.png', dpi=120, bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …